# Development Notebook

In this notebook we will develop the functions used in our Streamlit EV Adoption Tracker Application.

### 1. Load Data Function

In [1]:
import pandas as pd

In [2]:
ev_url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"

In [3]:
ev_df = pd.read_csv(ev_url)
ev_df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [4]:
# Loop through the columns and display all the unique classes for categorical columns, and basic stats for numerical columns
for col in ev_df.columns:
    print(f"Column: {col}")
    if ev_df[col].dtype == "object":
        print(f"{col}: {ev_df[col].unique()}")
    else:
        print(f"{col}: {ev_df[col].describe()}")
    print("\n")

Column: region
region: ['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']


Column: category
category: ['Historical']


Column: parameter
parameter: ['EV sales' 'EV stock share' 'EV sales share' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']


Column: mode
mode: ['Cars']


Column: powertrain
powertrain: ['BEV' 'EV' 'PHEV' 'FCEV']


Column: year
year: count    3798.000000
mean     2017.369932
std         3.809226
min      2010.000000
25%  

In [5]:
ev_df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [6]:
# Drop the columns that are not needed: category, mode, unit
ev_df = ev_df.drop(columns=["category", "mode", "unit"])

In [7]:
# Filter the dataframe to get the rows where parameter = "EV sales" and "EV sales share"
ev_sales_df = ev_df[ev_df["parameter"] == "EV sales"]
ev_sales_share_df = ev_df[ev_df["parameter"] == "EV sales share"]

# Drop the parameter column on both dataframes
ev_sales_df = ev_sales_df.drop(columns=["parameter"])
ev_sales_share_df = ev_sales_share_df.drop(columns=["parameter"])


In [11]:
# Drop the powertrain column
ev_sales_share_df = ev_sales_share_df.drop(columns=["powertrain"])


In [12]:
ev_sales_share_df.head()

,region,year,value
2,Australia,2011,0.0065
7,Australia,2012,0.0300
12,Australia,2013,0.0340
19,Australia,2014,0.1600
24,Australia,2015,0.2000


In [13]:
ev_sales_df.head()

,region,powertrain,year,value
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0


In [14]:
def load_data():
    ev_url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"
    ev_df = pd.read_csv(ev_url)
    # filter the dataframe to get the rows where parameter = "EV sales" and "EV sales share"
    ev_sales_df = ev_df[ev_df["parameter"] == "EV sales"]
    ev_sales_share_df = ev_df[ev_df["parameter"] == "EV sales share"]

    # Drop the unnecessary columns on both dataframes
    ev_sales_df = ev_sales_df.drop(columns=["category", "mode", "unit", "parameter"])
    ev_sales_share_df = ev_sales_share_df.drop(columns=["category", "mode", "unit", "parameter", "powertrain"])

    return ev_sales_df, ev_sales_share_df


In [18]:
ev_sales_df, ev_sales_share_df = load_data()
display(ev_sales_df.head())
display(ev_sales_share_df.head())

,region,powertrain,year,value
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0


,region,year,value
2,Australia,2011,0.0065
7,Australia,2012,0.0300
12,Australia,2013,0.0340
19,Australia,2014,0.1600
24,Australia,2015,0.2000


### Top Sales By Country Function

This function will take in the sales df and return a sorted dataframe with the top 10 countries with most sales.

In [19]:
ev_sales_df.head()

,region,powertrain,year,value
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0


In [23]:
# Filter to only include year 2023
ev_sales_df_2023 = ev_sales_df[ev_sales_df["year"] == 2023]

# Drop the year column
ev_sales_df_2023 = ev_sales_df_2023.drop(columns=["year"])

# Aggregate region sales by country
ev_sales_df_2023_agg = ev_sales_df_2023.groupby("region")["value"].sum().reset_index()

,region,value
0,Australia,98006.0
1,Austria,65009.0
2,Belgium,193009.0
3,Brazil,52000.0
4,Bulgaria,2160.0


In [24]:
# Rename the columns
ev_sales_df_2023_agg = ev_sales_df_2023_agg.rename(columns={"region": "Country", "value": "Sales"})

# Sort the dataframe by the sales column
ev_sales_df_2023_agg = ev_sales_df_2023_agg.sort_values(by="Sales", ascending=False)

# Display the top 10 countries with most sales
ev_sales_df_2023_agg.head(10)

,Country,Sales
51,World,13808900.0
7,China,8100520.0
16,Europe,3300820.0
14,EU27,2450770.0
48,USA,1393000.0
19,Germany,700260.0
18,France,470310.0
50,United Kingdom,450025.0
2,Belgium,193009.0
5,Canada,171013.0
